<a href="https://colab.research.google.com/github/yamazakikeisuke/python_for_journalism/blob/main/Python_for_journalism%EF%BC%88%E4%B8%8D%E5%8B%95%E7%94%A3%E3%81%AE%E5%8F%96%E5%BC%95%E4%BE%A1%E6%A0%BC%E5%88%86%E6%9E%90%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python for journalism用コード（土地情報編）
この１０年間で取引価格が高騰した土地を探してみる。

**１．データの読み込み**

In [ ]:
#テスト用のファイルをダウンロード
!wget  "https://drive.google.com/uc?export=download&id=1OP4MFrte4kG-qDlf4Zyzte6_acDeP2cR" -O 13_Tokyo_20111_20214.csv

In [ ]:
#ファイルを読み込んでみる
import pandas as pd
df = pd.read_csv('./13_Tokyo_20111_20214.csv')

In [ ]:
#３ＬＤＫのマンションにデータを絞る
df = df[df["間取り"] == "３ＬＤＫ"]
df = df[df["種類"] == "中古マンション等"]

In [ ]:
#バラバラの住所列をつなげた列を新たに作る
df["address"] = df['都道府県名'] + df['市区町村名'] + df['地区名']
#取引時点の列から「年」の情報だけを抜き出した列を作る
df["year"] = df["取引時点"].str[:4]
df["year"] = df["year"].astype(str)
#取引時点の列から「四半期」の情報だけを抜き出した列を作る
df["quater"] = df["取引時点"].str[5:]

In [ ]:
#分析で使う列名を設定
cols = ['都道府県名', '市区町村名', '地区名', '取引価格（総額）', '間取り', '種類', 'address', 'year']

In [ ]:
#各年・各地区の取引平均価格を計算
df_means = df.groupby(['都道府県名', '市区町村名', '地区名', 'address', '間取り', '種類', 'year'], as_index=False).mean()
df_means = df_means[cols]
#各年・各地区のデータ数を計算
df_size = df.groupby(['都道府県名', '市区町村名', '地区名', 'address', '間取り', '種類', 'year'], as_index=False).size()
df_size = df_size[['address', 'year', 'size']]

In [ ]:
#平均値とデータ数の計算結果を結合したdf_groupedを作る
df_grouped = pd.merge(df_means, df_size,  on=['address', 'year'], how='inner')
#各年でサンプル数が５件以上だった行のみを取り出す
df_grouped_over_5 = df_grouped[df_grouped['size'] >= 5]